In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers==4.8.2
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install torch 

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 29.1 MB/s 
     |████████████████████████████████| 3.3 MB 51.6 MB/s 
     |████████████████████████████████| 880 kB 49.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=fcdd676c511f91973f24aa3af8051ca75a63b542971b5e096c75e93afaf153af
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.9 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |█████

In [3]:

# torch
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
device = torch.device("cuda:0")

#BERT 모델, Vocabulary 불러오기 필수
bertmodel, vocab = get_pytorch_kobert_model()


# KoBERT에 입력될 데이터셋 정리
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))  

# 모델 정의
class BERTClassifier(nn.Module): ## 클래스를 상속
    def __init__(self,
                bert,
                hidden_size = 768,
                num_classes=7,   ##클래스 수 조정##
                dr_rate=None,
                params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# Setting parameters
max_len = 64
batch_size = 16
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

## 학습 모델 로드
PATH = '/content/drive/MyDrive/Colab Notebooks/Model/'
model = torch.load(PATH + 'KoBERT_model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def new_softmax(a) : 
    c = np.max(a) # 최댓값
    exp_a = np.exp(a-c) # 각각의 원소에 최댓값을 뺀 값에 exp를 취한다. (이를 통해 overflow 방지)
    sum_exp_a = np.sum(exp_a)
    y = (exp_a / sum_exp_a) * 100
    return np.round(y, 3)


# 예측 모델 설정
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        # test_eval=[]
        # for i in out:
        #     logits=i
        #     logits = logits.detach().cpu().numpy()
        #     min_v = min(logits)
        #     total = 0
        #     probability = []
        #     logits = np.round(new_softmax(logits), 3).tolist()
        #     for logit in logits:
        #         print(logit)
        #         probability.append(np.round(logit, 3))

          
    return out


/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [51]:
import warnings
import librosa
import librosa.display
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import sklearn
warnings.filterwarnings(action='ignore')

In [52]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [53]:
audioFilePath ='/content/drive/MyDrive/Colab Notebooks/Data/0001_G1A3E1S0C0_PSB_000001.wav'

In [54]:
voiceParam = extract_feature(audioFilePath,True,True,True)

In [55]:
voiceParam

array([-4.10504181e+02,  1.23570107e+02,  1.89433804e+01,  2.20322742e+01,
        6.74792528e+00,  1.63995590e+01, -1.06413708e+01,  5.17997026e+00,
        7.22772312e+00, -7.81177461e-01, -3.20249772e+00, -3.64560890e+00,
       -7.89813089e+00, -1.01306295e+01, -4.32525110e+00,  5.55748606e+00,
       -2.25704908e+00, -6.85884476e+00,  1.43046594e+00,  8.89175236e-01,
       -5.41176701e+00,  1.91487819e-01, -1.18871957e-01, -2.84854722e+00,
       -6.03678608e+00, -4.02946091e+00, -5.66906929e+00, -4.23324966e+00,
       -2.11053920e+00, -7.94421339e+00, -3.97826052e+00, -3.38989949e+00,
       -5.62452841e+00, -4.85505152e+00, -1.93656445e+00, -3.11400080e+00,
       -3.67292690e+00, -2.40853167e+00, -4.38236237e+00, -4.76423788e+00,
        6.50849760e-01,  6.55304909e-01,  6.31699026e-01,  6.43328309e-01,
        6.54490232e-01,  6.42884135e-01,  6.76011741e-01,  6.78369820e-01,
        6.72527373e-01,  6.97824180e-01,  7.02137589e-01,  6.48222625e-01,
        1.61888693e-02,  

In [4]:
from tensorflow.keras.models import load_model

In [5]:
SERmodel = load_model('/content/drive/MyDrive/Colab Notebooks/Model/voice_emotion_CnnModel1008.h5')
totalModel=load_model('/content/drive/MyDrive/Colab Notebooks/Model/concatModel10106.h5')

In [6]:
# 데이터 받아오는 알고리즘 추가

In [35]:
data = np.load('/content/drive/MyDrive/Colab Notebooks/Data/speechEmotion.npz')
X_train_SER=data['X_train']
y_train_SER=data['y_train']
y_test_SER=data['y_test']
X_test_SER=data['X_test']

In [34]:
data1 = np.load('/content/drive/MyDrive/Colab Notebooks/Data/sttEmotionTrainTest.npz')
X_train_STT=data1['X_train']


In [9]:
#--------------------------------

In [13]:
def softmax(a) :
  exp_a = np.exp(a)
  sum_exp_a = np.sum(exp_a)
  y=exp_a/sum_exp_a
  return y

In [44]:
def emotionReco(serData,terData):
  serData =serData.reshape(1,30, 6, 1)
  serResult =SERmodel.predict(serData)
  terResult =predict(str(terData))
  terResult=terResult.cpu().detach().numpy() 
  terResult=softmax(terResult)
  totalData = np.concatenate((serResult[0],terResult[0]))
  totalData =totalData.reshape(1,14)
  result = totalModel.predict(totalData)
  resultIndex =np.argmax(result[0])
  if resultIndex == 0:
    emotion = '기쁨'
  elif resultIndex==1:
    emotion = '슬픔'
  elif resultIndex==2:
    emotion = '분노'
  elif resultIndex==3:
    emotion = '불안'
  elif resultIndex==4:
    emotion = '상처'
  elif resultIndex==5:
    emotion = '당황'
  else :
    emotion = '중립'
  return emotion

In [48]:
emotionReco(X_train_SER[430000],X_train_STT[430000])

1/1 [==============================] - 0s 18ms/step


'중립'

In [50]:
# 감정 스프링으로 보내는 알고리즘 추가할것
# emotionReco 사용하면 감정 추출


In [26]:
from urllib import parse
from unicodedata import category
from flask import Flask,request,redirect,render_template
from sympy import re

app = Flask(__name__) # flask 서버객체 생성
 
# route(경로): 외부에서 접근할 때 페이지 구분을 위한 경로지정
@app.route('/')
def voice():
   
    # -- 텍스트 파일 
    # --- 음성파일 저장
    
    emotion = emotionReco(voiceParam, textParam)
    
    ## 코랩으로 자료 넘기기
        url =f"http://{ip}:8082/ikujo/RecommendFood.jsp?category={category}&{nutrients}{menu}{nutrientsReCo}"
 
    return redirect(url)

# main함수의 시작을 의미 
# 비유하자면 자바의 main(String[] args)메소드와 같은 역할
if __name__ == '__main__':
    # 웹서버주소(host)와 포트번호(port)를 지정하여 웹 서버 실행
    app.run(host='59.0.147.198', port='5000') 